# * Adhoc : Sales Performance

## Parameter

In [8]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
# from sqlalchemy import create_engine

In [9]:
config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2024, 6, 10)

## ETL Process

### Step 1 : ADHOC_RAW_SALE_PERFORMANCE_MTH
    Truncate -> Insert

In [ ]:
# Data Source

print(f'\nProcessing...')

query = """
    /*** Sales Performance Report ***/
    -----------------------------------------------------------------------------------------------------------------------

    WITH W_SALE_TXN AS 
    (
        SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, CENTER, PRODUCT_GRP, COMP_CD, METRIC_GRP, METRIC_NAME_GROUP, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
            , ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_SNAP, ACTUAL_AGG, TARGET_SNAP, TARGET_AGG--, BASELINE_SNAP, BASELINE_AGG, ACH_SNAP, ACH_AGG, GAP_SNAP, GAP_AGG
            , WOW, WOW_PERCENT, MOM, MOM_PERCENT, QOQ, QOQ_PERCENT, YOY, YOY_PERCENT--, RR, RR_ACH, WTD, MTD, QTD, YTD
            , PPN_TM
        FROM GEOSPCAPPO.AGG_PERF_NEWCO NOLOCK
        WHERE METRIC_GRP = 'Sales'
        AND REGEXP_LIKE(METRIC_CD, 'DB1R000900|DB2R000500|TB1R000900|TB2R000500')
        AND REGEXP_LIKE(METRIC_CD, '[0-9]$|[0-9]A[A-K]$')
        AND AREA_TYPE IN ('P', 'G', 'H')
        AND TM_KEY_YR >= 2023
        AND SUBSTR(TM_KEY_MTH,5,2) IN (01, 02, 03, 04)
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_SALE_ACTUAL_MONTHLY AS
    (
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
            , CASE WHEN AREA_TYPE = 'P' THEN 1 WHEN AREA_TYPE = 'G' THEN 2 WHEN AREA_TYPE = 'H' THEN 3 WHEN AREA_TYPE = 'HH' THEN 4 WHEN AREA_TYPE = 'Z' THEN 5 ELSE 0 END AREA_NO
            , AREA_TYPE, AREA_CD, AREA_NAME
            , SUM(ACTUAL_SNAP) AS ACTUAL_SNAP, SUM(TARGET_SNAP) AS TARGET_SNAP, MAX(PPN_TM) AS PPN_TM
        FROM W_SALE_TXN
        GROUP BY TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
    ) 
    -----------------------------------------------------------------------------------------------------------------------

    -->> Output

    SELECT *
    FROM W_SALE_ACTUAL_MONTHLY
    ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_TYPE, AREA_CD
"""

try:
    # TDMDBPR
    dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
    conn = oracledb.connect(dsn)
    print(f'\n{TDMDBPR_db} : Connected')
    cur = conn.cursor()

    # Create Rawdata
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=['TM_KEY_YR', 'TM_KEY_QTR', 'TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP', 'PPN_TM'])
    cur.close()
    df = df.replace(np.nan, None)
    df = df.astype(dtype={'TM_KEY_YR':'float', 'TM_KEY_QTR':'float', 'TM_KEY_MTH':'float', 'AREA_NO':'float'})
    rows = [tuple(x) for x in df.values]
    print(f'\n -> CREATE : Rawdata : Done !')


    # AKPIPRD
    op_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
    op_conn = oracledb.connect(op_dsn)
    print(f'\n{AKPIPRD_db} : Connected')
    op_cur = op_conn.cursor()

    # Truncate
    op_cur.execute("TRUNCATE TABLE AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH")
    print(f'\n -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !')

    # Insert
    op_cur.executemany("INSERT INTO ADHOC_RAW_SALE_PERFORMANCE_MTH\
                    (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM)\
                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14)", rows)
    op_cur.close()
    op_conn.commit()
    print(f'\n"-> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !\n\n{df.shape[0]} rows, {df.shape[1]} columns')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')
    op_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    print(f'\nJob Done !!!')


Processing...

TDMDBPR : Connected

 -> CREATE : Rawdata : Done !

AKPIPRD : Connected

 -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !

"-> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !

14262 rows, 14 columns

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


In [24]:
df.dtypes
# df[['PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates()

YR                          int64
MTH                        object
PRODUCT_GRP                object
METRIC_CD                  object
METRIC_NAME                object
AREA_NO                     int64
AREA_TYPE                  object
AREA_CD                    object
AREA_NAME                  object
ACHIEVEMENT               float64
MOM                       float64
YOY                       float64
SALE_PERF_COLOR            object
LAST_YR_ACTUAL            float64
LAST_MTH_ACTUAL           float64
ACTUAL                    float64
TARGET                    float64
T_ACTUAL                  float64
T_TARGET                  float64
D_ACTUAL                  float64
D_TARGET                  float64
LOAD_DATE          datetime64[ns]
dtype: object

#### Test : Generate Rawdata (CSV)

In [ ]:
# Source File

src_file = 'C:\Ruz\MyProject\Code\Jupyter\ETL\Adhoc\\rawdata\sales_performance_actual.csv'
src_df = pd.read_csv(src_file)
src_df = src_df.replace(np.nan, None)
src_df.head()
# rows = [tuple(x) for x in src_df.values]


# # AKPIPRD
# op_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
# op_conn = oracledb.connect(op_dsn)
# print(f'\n{AKPIPRD_db} : Connected')
# op_cur = op_conn.cursor()

# # Truncate
# # op_cur.execute("TRUNCATE TABLE AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH")
# print(f'\n -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH"')

# # Insert
# # op_cur.executemany("INSERT INTO ADHOC_RAW_SALE_PERFORMANCE_MTH (TM_KEY_YR) VALUES (:1)", rows)
# op_cur.executemany("INSERT INTO ADHOC_RAW_SALE_PERFORMANCE_MTH (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH) VALUES (:1,:2,:3)", rows)

# # op_cur.executemany("INSERT INTO ADHOC_RAW_SALE_PERFORMANCE_MTH (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM)\
# #                 VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14)", rows)

# print(f'\n -> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !')

# op_conn.commit()
# op_cur.close()
# op_conn.close()
# print(f'\n{AKPIPRD_db} : Disconnected')

,TM_KEY_YR,TM_KEY_QTR,TM_KEY_MTH,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,PPN_TM
0,2023,20231,202303,Postpaid,DTAC,DB2R000500,Postpaid Inflow M1 : DTAC,1,P,P,Nationwide,5.180456e+07,None,2024-04-23 12:22:16
1,2023,20231,202303,Postpaid,TRUE,TB2R000500,Postpaid Inflow M1 : TMH,1,P,P,Nationwide,6.901411e+07,96836673.93,2024-04-23 12:22:16
2,2023,20231,202303,Prepaid,DTAC,DB1R000900,Prepaid Inflow M1 : DTAC,1,P,P,Nationwide,1.511502e+08,None,2024-04-23 12:22:16
3,2023,20231,202303,Prepaid,TRUE,TB1R000900,Prepaid Inflow M1 : TMH,1,P,P,Nationwide,1.899225e+08,197205396.87,2024-04-23 12:22:16
4,2023,20232,202304,Postpaid,DTAC,DB2R000500,Postpaid Inflow M1 : DTAC,1,P,P,Nationwide,4.690888e+07,None,2024-04-23 12:22:16


In [ ]:
# TDMDBPR

# dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# conn = oracledb.connect(dsn)
# print(f'{TDMDBPR_db} : Connected')
# cur = conn.cursor()
# rawdata_dir = 'rawdata'
# rawdata_file = 'sales_performance_actual.csv'

# query = """
#     /*** Sales Growth Strategic report (Mar-Apr24) ***/
#     -----------------------------------------------------------------------------------------------------------------------

#     WITH W_SALE_TXN AS 
#     (
#         SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, CENTER, PRODUCT_GRP, COMP_CD, METRIC_GRP, METRIC_NAME_GROUP, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
#             , ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_SNAP, ACTUAL_AGG, TARGET_SNAP, TARGET_AGG--, BASELINE_SNAP, BASELINE_AGG, ACH_SNAP, ACH_AGG, GAP_SNAP, GAP_AGG
#             , WOW, WOW_PERCENT, MOM, MOM_PERCENT, QOQ, QOQ_PERCENT, YOY, YOY_PERCENT--, RR, RR_ACH, WTD, MTD, QTD, YTD
#             , PPN_TM
#         FROM GEOSPCAPPO.AGG_PERF_NEWCO NOLOCK
#         WHERE METRIC_GRP = 'Sales'
#         AND REGEXP_LIKE(METRIC_CD, 'DB1R000900|DB2R000500|TB1R000900|TB2R000500')
#         AND NOT REGEXP_LIKE(METRIC_CD, 'C$|H$|MCOM$|CORP$|GEO$') --A[A-K]$
#         AND AREA_TYPE IN ('P', 'G', 'H')
#         AND TM_KEY_YR >= 2023
#         AND SUBSTR(TM_KEY_MTH,5,2) IN (02, 03, 04)
#     )
#     -----------------------------------------------------------------------------------------------------------------------

#     , W_SALE_ACTUAL_MONTHLY AS
#     (
#         SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
#             , CASE WHEN AREA_TYPE = 'P' THEN 1 WHEN AREA_TYPE = 'G' THEN 2 WHEN AREA_TYPE = 'H' THEN 3 WHEN AREA_TYPE = 'HH' THEN 4 WHEN AREA_TYPE = 'Z' THEN 5 ELSE 0 END AREA_NO
#             , AREA_TYPE, AREA_CD, AREA_NAME
#             , SUM(ACTUAL_SNAP) AS ACTUAL_SNAP, SUM(TARGET_SNAP) AS TARGET_SNAP, MAX(PPN_TM) AS PPN_TM
#         FROM W_SALE_TXN
#         GROUP BY TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
#     ) 
#     -----------------------------------------------------------------------------------------------------------------------

#     -->> Output

#     SELECT *
#     FROM W_SALE_ACTUAL_MONTHLY
#     WHERE NOT REGEXP_LIKE(METRIC_CD, 'A[A-K]$')
#     AND AREA_TYPE = 'P'
#     ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_TYPE, AREA_CD
# """

# try:
#     print(f'\nProcessing...')
#     cur.execute(query)
#     df = pd.DataFrame(cur.fetchall(), columns=['TM_KEY_YR', 'TM_KEY_QTR', 'TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP', 'PPN_TM'])
#     # print(f'\n{df}')
#     df.to_csv(f'{rawdata_dir}/{rawdata_file}', index=False, encoding='utf-8')
#     print(f'\n -> "{rawdata_file}" is generated')

# except oracledb.DatabaseError as e:
#     print(f'Error with Oracle : {e}')

# finally:
#     cur.close()
#     conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

### Step 2 : Generate "Sales Performance Report"
    Excel file

In [21]:
# Output

print(f'\nProcessing...')

query = """
    /*** Sales Performance Report ***/
    -----------------------------------------------------------------------------------------------------------------------

    WITH W_SALE_TARGET_MONTHLY AS 
    (
        SELECT TM_KEY_MTH, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, SUM(METRIC_VALUE) AS TARGET_MTH
        
        FROM AUTOKPI.ADHOC_DTAC_TARGET_AREA_DAILY
        
        WHERE METRIC_GRP = 'Sales'
        AND REGEXP_LIKE(METRIC_CD, 'DB1R000900|DB2R000500|TB1R000900') --|TB2R000500
        AND REGEXP_LIKE(METRIC_CD, '[0-9]$|[0-9]A[A-K]$')
        
        GROUP BY TM_KEY_MTH, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_ADJUST_TARGET_DTAC AS 
    (
        SELECT A.TM_KEY_YR, A.TM_KEY_QTR, A.TM_KEY_MTH, A.PRODUCT_GRP, A.COMP_CD, A.METRIC_CD, A.METRIC_NAME
            , REGEXP_REPLACE(A.METRIC_CD, 'DB|TB', 'B') AS TMP_METRIC_CD
            , REGEXP_REPLACE(A.METRIC_NAME, ' : DTAC| : TMH') AS TMP_METRIC_NAME
            , CASE WHEN REGEXP_LIKE(SUBSTR(A.METRIC_CD,-2), 'A[A-Z]$') THEN SUBSTR(A.METRIC_CD,-2) ELSE 'ALL' END CHANNEL_CD
            , A.AREA_NO, A.AREA_TYPE, A.AREA_CD, A.AREA_NAME
            , COALESCE(A.ACTUAL_SNAP, 0) AS ACTUAL_SNAP
            , COALESCE(CASE WHEN T.TARGET_MTH IS NOT NULL THEN T.TARGET_MTH
                            ELSE A.TARGET_SNAP
                            END, 0) AS TARGET_SNAP
            , A.PPN_TM, A.LOAD_DATE
        
        FROM AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH A
        
        LEFT JOIN W_SALE_TARGET_MONTHLY T
            ON A.TM_KEY_MTH = T.TM_KEY_MTH
            AND A.METRIC_CD = T.METRIC_CD
            AND A.AREA_CD = T.AREA_CD
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_TOTAL_AGG AS 
    (	
        -->> ALL = (TRUE + DTAC)
        SELECT A.*
            , T.ACTUAL_SNAP AS T_ACTUAL, T.TARGET_SNAP AS T_TARGET
            , D.ACTUAL_SNAP AS D_ACTUAL, D.TARGET_SNAP AS D_TARGET
        
        FROM (
            SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH
                , SUBSTR(TM_KEY_MTH, 5, 2) AS MTH
                , PRODUCT_GRP
                , 'ALL' AS COMP_CD, TMP_METRIC_CD AS METRIC_CD, TMP_METRIC_NAME AS METRIC_NAME
                , CHANNEL_CD, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
                , SUM(ACTUAL_SNAP) AS ACTUAL, SUM(TARGET_SNAP) AS TARGET
                , MAX(PPN_TM) AS PPN_TM, MAX(LOAD_DATE) AS LOAD_DATE
            FROM W_ADJUST_TARGET_DTAC
            GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, TMP_METRIC_CD, TMP_METRIC_NAME, CHANNEL_CD, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
        ) A
        
        LEFT JOIN ( -->> TRUE
            SELECT TM_KEY_MTH, COMP_CD, TMP_METRIC_CD, TMP_METRIC_NAME, CHANNEL_CD, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP
            FROM W_ADJUST_TARGET_DTAC
            WHERE COMP_CD = 'TRUE'
        ) T
            ON A.TM_KEY_MTH = T.TM_KEY_MTH
            AND A.METRIC_CD = T.TMP_METRIC_CD
            AND A.AREA_CD = T.AREA_CD
            
        LEFT JOIN ( -->> DTAC
            SELECT TM_KEY_MTH, COMP_CD, TMP_METRIC_CD, TMP_METRIC_NAME, CHANNEL_CD, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP
            FROM W_ADJUST_TARGET_DTAC
            WHERE COMP_CD = 'DTAC'
        ) D
            ON A.TM_KEY_MTH = D.TM_KEY_MTH
            AND A.METRIC_CD = D.TMP_METRIC_CD
            AND A.AREA_CD = D.AREA_CD
    )
    -----------------------------------------------------------------------------------------------------------------------

    -->> Output

    SELECT TM_KEY_YR AS YR, MTH, PRODUCT_GRP, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACHIEVEMENT, MOM, YOY
        -->> Sales Performance Color
        , CASE 	WHEN ACTUAL > TARGET THEN 'green'
                WHEN ACTUAL < TARGET AND ACTUAL > LAST_YR_ACTUAL THEN 'yellow'
                WHEN ACTUAL < TARGET AND ACTUAL < LAST_YR_ACTUAL THEN 'red'
                END SALE_PERF_COLOR
        , LAST_YR_ACTUAL, LAST_MTH_ACTUAL, ACTUAL, TARGET, T_ACTUAL, T_TARGET, D_ACTUAL, D_TARGET, LOAD_DATE
        
    FROM (
        SELECT TM_KEY_YR, TM_KEY_MTH, MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, CHANNEL_CD, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
            -->> % Achievement
            , CASE 	WHEN COALESCE(ACTUAL,0) = 0 OR COALESCE(TARGET,0) = 0 THEN NULL 
                    ELSE ACTUAL / TARGET * 100 END ACHIEVEMENT
            -->> % MoM
            , CASE 	WHEN COALESCE(ACTUAL,0) = 0 OR COALESCE(LAST_MTH_ACTUAL,0) = 0 THEN NULL 
                    ELSE (ACTUAL - LAST_MTH_ACTUAL) / LAST_MTH_ACTUAL * 100 END MOM
            -->> % YoY
            , CASE 	WHEN COALESCE(ACTUAL,0) = 0 OR COALESCE(LAST_YR_ACTUAL,0) = 0 THEN NULL 
                    ELSE (ACTUAL - LAST_YR_ACTUAL) / LAST_YR_ACTUAL * 100 END YOY
            , LAST_YR_ACTUAL, LAST_MTH_ACTUAL, ACTUAL, TARGET, T_ACTUAL, T_TARGET, D_ACTUAL, D_TARGET, LOAD_DATE
            
        FROM (
            SELECT TM_KEY_YR, TM_KEY_MTH, MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, CHANNEL_CD, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
                , LAG(ACTUAL) OVER(PARTITION BY METRIC_CD, AREA_CD, MTH ORDER BY TM_KEY_YR) LAST_YR_ACTUAL
                , LAG(ACTUAL) OVER(PARTITION BY METRIC_CD, AREA_CD, TM_KEY_YR ORDER BY TM_KEY_MTH) LAST_MTH_ACTUAL
                , ACTUAL, TARGET, T_ACTUAL, T_TARGET, D_ACTUAL, D_TARGET, LOAD_DATE
            FROM W_TOTAL_AGG
        ) AGG_1
        
    ) AGG_2

    WHERE TM_KEY_MTH BETWEEN 202403 AND 202404

    ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_NO, AREA_CD 
"""


# AKPIPRD

dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected')
cur = conn.cursor()
op_dir = 'output'
op_file = 'sales_performance.xlsx'


try:
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=['YR', 'MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACHIEVEMENT', 'MOM', 'YOY', 'SALE_PERF_COLOR', 'LAST_YR_ACTUAL', 'LAST_MTH_ACTUAL', 'ACTUAL', 'TARGET', 'T_ACTUAL', 'T_TARGET', 'D_ACTUAL', 'D_TARGET', 'LOAD_DATE'])
    # print(f'\n{df}')
    df.to_excel(f'{op_dir}/{op_file}', sheet_name='Data', index=False)
    print(f'\n -> Generate "{op_file}" successfully')


except oracledb.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')


Processing...

AKPIPRD : Connected

 -> Generate "sales_performance.xlsx" successfully

AKPIPRD : Disconnected
